# 001 Preprocess and analyse ECG

### Purpose
Purpose of this notebook is to preprocess the raw ECG data for the 'We Love Reading' study.

- Neurokit2 package with a custom pipeline will be used for preprocessing
- Segmentation is performed afterwards
- QA figures will be exported for every partipant
- The processed signal data will be exported
- HRV metrics are computed and exported


### Input / Output
- Input: `~/data/raw`
- Outputs:
  - QA visualizations: `~/reports/ECG QA`
  - Processed signal data and HRV metrics: `~/data/processed`
  
**Note**:
- Output folders are created if they do not exist


### Imports

In [26]:
# fmt: off
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import importlib
import nk_pipeline
import neurokit2 as nk
import parameters as params
import data_utils
import common
importlib.reload(nk)
importlib.reload(nk_pipeline)
importlib.reload(params)
importlib.reload(data_utils)
importlib.reload(common)

# fmt:on


<module 'common' from '/Users/lukasspiess/Library/CloudStorage/OneDrive-SpiessSolution/Documents - We Love Reading/Welcome/notebooks/common.py'>

## Parameters

#### IO

In [27]:
WORKING_DIR = Path().cwd()
ROOT_DIR = WORKING_DIR.parent
DATA_DIR = ROOT_DIR / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
PROCESSED_DATA_DIR.mkdir(parents=False, exist_ok=True)
REPORTS_DIR = ROOT_DIR / 'reports'
REPORTS_DIR.mkdir(parents=False, exist_ok=True) 
QA_REPORTS_DIR = REPORTS_DIR / 'QA'
QA_REPORTS_DIR.mkdir(parents=False, exist_ok=True) 

## Support Functions

# Prepare Data

## Filepaths

ECG

In [28]:
ecg_filepaths = np.sort(list(RAW_DATA_DIR.glob('*mc.txt')))
# [print(f) for f in ecg_filepaths];
[print(data_utils.extract_subject_id_condition_from_filepath(f)) for f in ecg_filepaths];

(8, 'B')
(32, 'B')
(8, 'C')
(32, 'C')
(8, 'W')
(32, 'W')


Events

In [29]:
event_filepaths = np.sort(list(RAW_DATA_DIR.glob('*event.txt')))
# [print(f) for f in event_filepaths];
[print(data_utils.extract_subject_id_condition_from_filepath(f)) for f in event_filepaths];

(8, 'B')
(32, 'B')
(8, 'C')
(32, 'C')
(8, 'W')
(32, 'W')


# Apply the pipeline

- Make sure that the order of the ECG and Event files match.

**ToDo**
- Improve reading/checking ECG and Event files to make sure they match

In [ ]:
for index in range(len(ecg_filepaths))[:10]:
    dyad_number, condition = data_utils.extract_subject_id_condition_from_filepath(ecg_filepaths[index])
    print(f"Processing recording {index+1}/{len(ecg_filepaths)}. Dyad number: {dyad_number}. Condition: {condition}")
    
    nk_pipeline.process_dyad(
        ecg_filepath = ecg_filepaths[index],
        event_filepath = event_filepaths[index],
        parameters=params.base_params,
        data_output_dir=PROCESSED_DATA_DIR,
        figure_output_dir=QA_REPORTS_DIR
    );

# DEBUG STUFF
|    
V    

# Preprocess first

In [ ]:
# Load and prepare
signal_event_df = nk_pipeline.load_dyad_ecg_events(ecg_filepaths[0], event_filepaths[0])
dyad_parameters = params.configure_segmentation_params(1, params.base_params)
child_params, mother_params = params.configure_ecg_params(1, dyad_parameters)
child_series, mother_series = nk_pipeline.split_in_child_mother_series(signal_event_df)

# Preprocess ECG
child_signals_df = nk_pipeline.ecg_preprocess(child_series, child_params)
mother_signals_df = nk_pipeline.ecg_preprocess(mother_series, mother_params)

# Join the preprocessed signals with the events
child_signal_event_df = child_signals_df.merge(signal_event_df[["event", "event_description"]], left_index=True, right_index=True, how = "left")
mother_signal_event_df = mother_signals_df.merge(signal_event_df[["event", "event_description"]], left_index=True, right_index=True, how = "left")

In [ ]:
child_signal_event_df.head()

## Then segment and analyze

In [ ]:
# segment the dataframe
child_segments_df_list = nk_pipeline.segment_df(child_signal_event_df, dyad_parameters)
mother_segments_df_list = nk_pipeline.segment_df(mother_signal_event_df, dyad_parameters)

In [ ]:
child_segments_df_list[0]

In [ ]:
child_segments_df_list[0]["event_description"].iloc[0]